In [48]:
import sys
sys.path.append("../model/robosat_pink/")
from robosat_pink import datasets
from robosat_pink.config import load_config

import os 
os.environ['CURL_CA_BUNDLE']='/etc/ssl/certs/ca-certificates.crt'

import pandas as pd
from pandas import Series

import boto3
import s3fs
from re import match
import tqdm
from itertools import chain
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio import Env

from multiprocessing import Pool

import numpy as np

import ipywidgets as widgets

%matplotlib inline

## Clean Bad Tiles

We're having an issue where lots of image tiles have no data. Let's clean them up. 


In [78]:
tilefolder_re = ".*_tiled"
tile_re = ".*_tiled.*\/\d*\/\d*\/\d*\.tif"

In [92]:
fs = s3fs.S3FileSystem(session = boto3.Session(profile_name = 'esip'))
tileFolderCandidates = fs.ls('planet-snowcover-imagery', update=True)

In [93]:
tileFolders = [f for f in tileFolderCandidates if match(tilefolder_re, f)]

In [94]:
allTiles = []
for folder in tqdm.tqdm(tileFolders, unit='folder'):
    allfiles = fs.walk(folder)
    tiles = [t for t in allfiles if match(tile_re, t)]
    allTiles.append(tiles)
allTiles = list(chain(*allTiles))

100%|██████████| 16/16 [00:05<00:00,  2.76folder/s]


In [97]:
allTiles

['planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5528/12722.tif',
 'planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5529/12721.tif',
 'planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5529/12722.tif',
 'planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5530/12720.tif',
 'planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5530/12721.tif',
 'planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5530/12722.tif',
 'planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5530/12723.tif',
 'planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5531/12720.tif',
 'planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5531/12721.tif',
 'planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5531/12722.tif',
 'planet-snowcover-imagery/20180422_1808

---

In [57]:
def partialTile(tilepath):
    with rio.Env(profile_name='esip'):
        try: 
            data = rio.open('s3://' + tilepath).read()
        except rio.errors.RasterioIOError as e:
            print(e)
            return False 
        
    flat = data.flatten()
    return flat.shape[0] != np.count_nonzero(flat)


        

In [68]:
len(allTiles)

2555

In [58]:
p = Pool(processes = 10)
result = p.map(partialTile, allTiles)

'/vsis3/planet-snowcover-imagery/20180601_181447_0f32_3B_AnalyticMS_SR_clip_tiled/15/5543/12712.tif.aux.xml' not recognized as a supported file format.
'/vsis3/planet-snowcover-imagery/20180601_181447_0f32_3B_AnalyticMS_SR_clip_tiled/15/5548/12703.tif.aux.xml' not recognized as a supported file format.
'/vsis3/planet-snowcover-imagery/20180601_181447_0f32_3B_AnalyticMS_SR_clip_tiled/15/5560/12706.tif.aux.xml' not recognized as a supported file format.
'/vsis3/planet-snowcover-imagery/20180601_181448_0f32_3B_AnalyticMS_SR_clip_tiled/15/5541/12709.tif.aux.xml' not recognized as a supported file format.
'/vsis3/planet-snowcover-imagery/20180601_181450_0f32_3B_AnalyticMS_SR_clip_tiled/15/5540/12691.tif.aux.xml' not recognized as a supported file format.
'/vsis3/planet-snowcover-imagery/20180601_181448_0f32_3B_AnalyticMS_SR_clip_tiled/15/5547/12702.tif.aux.xml' not recognized as a supported file format.
'/vsis3/planet-snowcover-imagery/20180601_181448_0f32_3B_AnalyticMS_SR_clip_tiled/15/554

Process ForkPoolWorker-19:
Process ForkPoolWorker-18:
Process ForkPoolWorker-11:
Process ForkPoolWorker-16:
Process ForkPoolWorker-15:
Process ForkPoolWorker-13:
Traceback (most recent call last):
Process ForkPoolWorker-12:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-17:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Process ForkPoolWorker-14:
Process ForkPoolWorker-20:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/multiprocessing/process.py", line 258, in _bo

In [63]:
partialTiles = tiles[Series(result)]

In [145]:
def mvPartialTile(tile):
    base, ext = os.path.splitext(tile)
    newBase = base+'_partialtile'
    fs.mv(tile, newBase + ext)

In [146]:
errored = 0
for tile in tqdm.tqdm(partialTiles, unit='tile'):
    try: 
        mvPartialTile(tile)
    except Exception as e: 
        errored += 1
        continue
        
print(errored)

    

100%|██████████| 1486/1486 [14:35<00:00,  1.70tile/s]

5


In [87]:
partialTiles[0]

'planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5526/12722.tif'

In [90]:
with rio.Env(profile_name='esip'):
    rio.open("s3://planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5526/12722_partialtile.tif").read()

---

In [47]:
completeTiles.to_csv(fs.open('planet-snowcover-imagery/complete-tiles.csv', 'w'))

In [13]:

completeTiles = pd.read_csv(fs.open("planet-snowcover-imagery/complete-tiles.csv", 'r'), header=None, index_col=0).values

---
# Diagnostics

In [104]:
# should have some partial tiles, we don't want 'em
tiledir = "planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled"
maskdir = "planet-snowcover-snow/ASO_3M_SD_USCAJW_20180423/ASO_3M_SD_USCAJW_20180423-MASK_02-COG"

In [102]:
from robosat_pink.datasets import MultiSlippyMapTilesConcatenation

In [105]:
mt = MultiSlippyMapTilesConcatenation([tiledir], [maskdir], aws_profile='esip')

In [112]:

partialTile('planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5540/12721.tif')

True

In [128]:
list(partialTiles.values)

['planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5526/12722.tif',
 'planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5527/12720.tif',
 'planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5527/12721.tif',
 'planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5527/12722.tif',
 'planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5528/12721.tif',
 'planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5528/12722.tif',
 'planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5529/12721.tif',
 'planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5529/12722.tif',
 'planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5530/12720.tif',
 'planet-snowcover-imagery/20180422_180809_1033_3B_AnalyticMS_SR_clip_tiled/15/5530/12721.tif',
 'planet-snowcover-imagery/20180422_1808